In [1]:
import pandas as pd
import pandas as pd
from collections import defaultdict
from sklearn import metrics
import numpy as np
import sys
import time
import os

start_time = time.time()

inbinfile='/Users/irffanalahi/Research/Research_update/SoftRD/using13cref/HardRC/BLthirteen_all_matrixCin_nr0.5_imputed_rowmean_bg_inter_t1.15t2.15_othermean.txt_result_dupindex_binnedstats.pkl'


outfolder=inbinfile+"_HardRC"
os.mkdir(outfolder)

outfile=outfolder+"/"+os.path.basename(inbinfile)


#######outfile=inbinfile


mincpgLIST=[1,2,3]

consideringALLheyper=True


inbindf=pd.read_pickle(inbinfile)



#scorecolumns=['CD4-others','CD8-others','nB-others','NK-others','Mn-others','mNeu-others','m8-others','DC-others','Eo-others','Tr-others','m4-others','Mg-others','Er-others','mB-others']


#13ctref
scorecolumns=['CD4-others','CD8-others','nB-others','NK-others','PC-others','Mn-others','iDC-others','mDC-others','M1-others','M0-others','mN-others','M2-others','cB-others']

inbindf=inbindf.reset_index()
inbindf.head()

,index,CD4-others,CD8-others,nB-others,NK-others,PC-others,Mn-others,iDC-others,mDC-others,M1-others,...,LENhypoCpG,LENhyperCpG,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename
0,A00354:332:HKTJ2DSXY:4:1101:10185:35415,0.176208,0.384931,-0.088125,0.119243,-0.083340,-0.079007,-0.053729,-0.048944,-0.049035,...,2.0,6.0,8.0,0.250000,CD8-others,1.539722,CD8-others,0.384931,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
1,A00354:332:HKTJ2DSXY:4:1101:17662:5431,-0.052000,0.271556,-0.036352,-0.020583,0.029731,-0.030093,-0.051157,-0.045620,0.001926,...,1.0,2.0,3.0,0.333333,CD8-others,0.814667,CD8-others,0.271556,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
2,A00354:332:HKTJ2DSXY:4:1101:25274:21308,-0.019378,-0.015181,-0.016715,0.000708,0.011767,-0.005431,-0.009177,-0.015767,0.007795,...,1.0,7.0,8.0,0.125000,M2-others,0.470417,M2-others,0.058802,M2-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
3,A00354:332:HKTJ2DSXY:4:1101:26006:9643,-0.025563,-0.063660,0.412014,0.081687,0.016146,-0.092007,-0.057521,-0.069618,-0.092007,...,1.0,1.0,2.0,0.500000,nB-others,0.824028,nB-others,0.412014,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
4,A00354:332:HKTJ2DSXY:4:1101:26467:17331,-0.092843,-0.095250,1.084139,-0.118843,-0.113787,-0.158324,-0.127750,-0.129917,-0.134731,...,2.0,1.0,3.0,0.666667,nB-others,1.626208,nB-others,1.084139,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...


In [2]:
if consideringALLheyper==True:
    allassigned=inbindf.copy()


elif consideringALLheyper==False:
    allassigned=inbindf[inbindf['deltabasedfragassignment']!='NotAssigned'].copy()
    
else:
    print("wrong input")
    sys.exit(1)
#
allassigned.head()

,index,CD4-others,CD8-others,nB-others,NK-others,PC-others,Mn-others,iDC-others,mDC-others,M1-others,...,LENhypoCpG,LENhyperCpG,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename
0,A00354:332:HKTJ2DSXY:4:1101:10185:35415,0.176208,0.384931,-0.088125,0.119243,-0.083340,-0.079007,-0.053729,-0.048944,-0.049035,...,2.0,6.0,8.0,0.250000,CD8-others,1.539722,CD8-others,0.384931,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
1,A00354:332:HKTJ2DSXY:4:1101:17662:5431,-0.052000,0.271556,-0.036352,-0.020583,0.029731,-0.030093,-0.051157,-0.045620,0.001926,...,1.0,2.0,3.0,0.333333,CD8-others,0.814667,CD8-others,0.271556,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
2,A00354:332:HKTJ2DSXY:4:1101:25274:21308,-0.019378,-0.015181,-0.016715,0.000708,0.011767,-0.005431,-0.009177,-0.015767,0.007795,...,1.0,7.0,8.0,0.125000,M2-others,0.470417,M2-others,0.058802,M2-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
3,A00354:332:HKTJ2DSXY:4:1101:26006:9643,-0.025563,-0.063660,0.412014,0.081687,0.016146,-0.092007,-0.057521,-0.069618,-0.092007,...,1.0,1.0,2.0,0.500000,nB-others,0.824028,nB-others,0.412014,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...
4,A00354:332:HKTJ2DSXY:4:1101:26467:17331,-0.092843,-0.095250,1.084139,-0.118843,-0.113787,-0.158324,-0.127750,-0.129917,-0.134731,...,2.0,1.0,3.0,0.666667,nB-others,1.626208,nB-others,1.084139,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...


In [3]:
allassigned.loc[inbindf['deltabasedfragassignment']=='NotAssigned','deltabasedfragassignment']=allassigned.loc[inbindf['deltabasedfragassignment']=='NotAssigned','maxscoredCT_beforeCpGweight']

In [4]:
allassigned['secondmaxScore']='Notassigned'
allassigned['secondmaxScoreCT']='Notassigned'
allassigned.head()

,index,CD4-others,CD8-others,nB-others,NK-others,PC-others,Mn-others,iDC-others,mDC-others,M1-others,...,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT
0,A00354:332:HKTJ2DSXY:4:1101:10185:35415,0.176208,0.384931,-0.088125,0.119243,-0.083340,-0.079007,-0.053729,-0.048944,-0.049035,...,8.0,0.250000,CD8-others,1.539722,CD8-others,0.384931,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,Notassigned,Notassigned
1,A00354:332:HKTJ2DSXY:4:1101:17662:5431,-0.052000,0.271556,-0.036352,-0.020583,0.029731,-0.030093,-0.051157,-0.045620,0.001926,...,3.0,0.333333,CD8-others,0.814667,CD8-others,0.271556,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,Notassigned,Notassigned
2,A00354:332:HKTJ2DSXY:4:1101:25274:21308,-0.019378,-0.015181,-0.016715,0.000708,0.011767,-0.005431,-0.009177,-0.015767,0.007795,...,8.0,0.125000,M2-others,0.470417,M2-others,0.058802,M2-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,Notassigned,Notassigned
3,A00354:332:HKTJ2DSXY:4:1101:26006:9643,-0.025563,-0.063660,0.412014,0.081687,0.016146,-0.092007,-0.057521,-0.069618,-0.092007,...,2.0,0.500000,nB-others,0.824028,nB-others,0.412014,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,Notassigned,Notassigned
4,A00354:332:HKTJ2DSXY:4:1101:26467:17331,-0.092843,-0.095250,1.084139,-0.118843,-0.113787,-0.158324,-0.127750,-0.129917,-0.134731,...,3.0,0.666667,nB-others,1.626208,nB-others,1.084139,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,Notassigned,Notassigned


In [5]:
grouped=allassigned.groupby('maxscoredCT')

for name, group in grouped:
    
    
    tempscorecolumns=scorecolumns.copy()
    
    tempscorecolumns.remove(name)
    
    allassigned.loc[group.index,'secondmaxScore']=group[tempscorecolumns].max(axis=1)
    allassigned.loc[group.index,'secondmaxScoreCT']=(group[tempscorecolumns]).idxmax(axis=1)

In [6]:
allassigned['adjustedScore_minus']=allassigned['maxscore']-allassigned['secondmaxScore']
allassigned['adjustedScore_minus_avg']=(allassigned['maxscore']+(allassigned['maxscore']-allassigned['secondmaxScore']))/2
allassigned.head()

,index,CD4-others,CD8-others,nB-others,NK-others,PC-others,Mn-others,iDC-others,mDC-others,M1-others,...,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT,adjustedScore_minus,adjustedScore_minus_avg
0,A00354:332:HKTJ2DSXY:4:1101:10185:35415,0.176208,0.384931,-0.088125,0.119243,-0.083340,-0.079007,-0.053729,-0.048944,-0.049035,...,CD8-others,1.539722,CD8-others,0.384931,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.176208,CD4-others,0.208722,0.296826
1,A00354:332:HKTJ2DSXY:4:1101:17662:5431,-0.052000,0.271556,-0.036352,-0.020583,0.029731,-0.030093,-0.051157,-0.045620,0.001926,...,CD8-others,0.814667,CD8-others,0.271556,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0297315,PC-others,0.241824,0.25669
2,A00354:332:HKTJ2DSXY:4:1101:25274:21308,-0.019378,-0.015181,-0.016715,0.000708,0.011767,-0.005431,-0.009177,-0.015767,0.007795,...,M2-others,0.470417,M2-others,0.058802,M2-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0191701,M0-others,0.0396319,0.049217
3,A00354:332:HKTJ2DSXY:4:1101:26006:9643,-0.025563,-0.063660,0.412014,0.081687,0.016146,-0.092007,-0.057521,-0.069618,-0.092007,...,nB-others,0.824028,nB-others,0.412014,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0816875,NK-others,0.330326,0.37117
4,A00354:332:HKTJ2DSXY:4:1101:26467:17331,-0.092843,-0.095250,1.084139,-0.118843,-0.113787,-0.158324,-0.127750,-0.129917,-0.134731,...,nB-others,1.626208,nB-others,1.084139,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.144769,cB-others,0.93937,1.01175


In [7]:
allassigned.columns

Index(['index', 'CD4-others', 'CD8-others', 'nB-others', 'NK-others',
       'PC-others', 'Mn-others', 'iDC-others', 'mDC-others', 'M1-others',
       'M0-others', 'mN-others', 'M2-others', 'cB-others', 'LENhypoCpG',
       'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg',
       'maxscoredCT_beforeCpGweight', 'maxscore_beforeCpGweight',
       'maxscoredCT', 'maxscore', 'deltabasedfragassignment', 'filename',
       'secondmaxScore', 'secondmaxScoreCT', 'adjustedScore_minus',
       'adjustedScore_minus_avg'],
      dtype='object')

In [8]:
adjustedScore_minus0val=list(set(allassigned[allassigned['LENhypoCpG_BY_total_cpg']==0]['adjustedScore_minus'].tolist()))
if len(adjustedScore_minus0val)==0:
    pass
elif len(adjustedScore_minus0val)!=1 or adjustedScore_minus0val[0]!=0:
    print(adjustedScore_minus0val)
    sys.exit(1)
    

In [9]:
adjustedScore_minus_avg0val=list(set(allassigned[allassigned['LENhypoCpG_BY_total_cpg']==0]['adjustedScore_minus_avg'].tolist()))


if len(adjustedScore_minus_avg0val)==0:
    pass

elif len(adjustedScore_minus_avg0val)!=1 or adjustedScore_minus_avg0val[0]!=0:
    print(adjustedScore_minus_avg0val)
    sys.exit(1)
    

In [10]:
allassigned['LENhypoCpG_0.8']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.8,'LENhypoCpG_0.8']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.8,'LENhypoCpG']

In [11]:
allassigned['LENhypoCpG_0.65']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.65,'LENhypoCpG_0.65']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.65,'LENhypoCpG']

In [12]:
allassigned['LENhypoCpG_0.5']=0
allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.5,'LENhypoCpG_0.5']=allassigned.loc[allassigned['LENhypoCpG_BY_total_cpg']>=0.5,'LENhypoCpG']

In [13]:
allassigned.head()

,index,CD4-others,CD8-others,nB-others,NK-others,PC-others,Mn-others,iDC-others,mDC-others,M1-others,...,maxscore,deltabasedfragassignment,filename,secondmaxScore,secondmaxScoreCT,adjustedScore_minus,adjustedScore_minus_avg,LENhypoCpG_0.8,LENhypoCpG_0.65,LENhypoCpG_0.5
0,A00354:332:HKTJ2DSXY:4:1101:10185:35415,0.176208,0.384931,-0.088125,0.119243,-0.083340,-0.079007,-0.053729,-0.048944,-0.049035,...,0.384931,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.176208,CD4-others,0.208722,0.296826,0.0,0.0,0.0
1,A00354:332:HKTJ2DSXY:4:1101:17662:5431,-0.052000,0.271556,-0.036352,-0.020583,0.029731,-0.030093,-0.051157,-0.045620,0.001926,...,0.271556,CD8-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0297315,PC-others,0.241824,0.25669,0.0,0.0,0.0
2,A00354:332:HKTJ2DSXY:4:1101:25274:21308,-0.019378,-0.015181,-0.016715,0.000708,0.011767,-0.005431,-0.009177,-0.015767,0.007795,...,0.058802,M2-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0191701,M0-others,0.0396319,0.049217,0.0,0.0,0.0
3,A00354:332:HKTJ2DSXY:4:1101:26006:9643,-0.025563,-0.063660,0.412014,0.081687,0.016146,-0.092007,-0.057521,-0.069618,-0.092007,...,0.412014,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.0816875,NK-others,0.330326,0.37117,0.0,0.0,1.0
4,A00354:332:HKTJ2DSXY:4:1101:26467:17331,-0.092843,-0.095250,1.084139,-0.118843,-0.113787,-0.158324,-0.127750,-0.129917,-0.134731,...,1.084139,nB-others,mel-159-P-Bulk_sorted_onlyregionIN_data13ct_hy...,0.144769,cB-others,0.93937,1.01175,0.0,2.0,2.0


In [14]:
def hardRC(fgrouped,targetmincpg,cpgpatternratecolumn,fname):
    ctproportiondict=defaultdict(list)
    for score in scorecolumns:
        
     
        
        totalfragments=fgrouped[(fgrouped['deltabasedfragassignment']==score) & (fgrouped['total_cpg']>=targetmincpg)].shape[0]
        positivefragments=fgrouped[(fgrouped['deltabasedfragassignment']==score) & (fgrouped[cpgpatternratecolumn]>=targetmincpg)].shape[0]
        

        

        
        if totalfragments==0:
            tempctproportion=0
        
        else:
            tempctproportion=1.0*positivefragments/totalfragments
        
        ctname=score.replace('-others','')
        
        ctproportiondict[ctname].append(tempctproportion)
        
    ctproportionDF=pd.DataFrame.from_dict(ctproportiondict)
    
    ctproportionDF['Mixture']=fname
        
    return  ctproportionDF
    
  

In [15]:
filegrouped=allassigned.groupby('filename')

for mincpg in mincpgLIST:
    allfileresultfiftypercent=[]
    allfileresultsixtyfivepercent=[]
    allfileresulteightypercent=[]
    for name, group in filegrouped:
        allfileresultfiftypercent.append(hardRC(group,mincpg,'LENhypoCpG_0.5',name))
        allfileresultsixtyfivepercent.append(hardRC(group,mincpg,'LENhypoCpG_0.65',name))
        allfileresulteightypercent.append(hardRC(group,mincpg,'LENhypoCpG_0.8',name))
        
        
        
     

    allfileresultfiftypercentDF=pd.concat(allfileresultfiftypercent)
    allfileresultfiftypercentDF.set_index('Mixture',inplace=True) 
    
    allfileresultsixtyfivepercentDF=pd.concat(allfileresultsixtyfivepercent)
    allfileresultsixtyfivepercentDF.set_index('Mixture',inplace=True) 
    
    
    allfileresulteightypercentDF=pd.concat(allfileresulteightypercent)
    allfileresulteightypercentDF.set_index('Mixture',inplace=True)
    
    
    
    allfileresultfiftypercentDF.to_csv(outfile+"_pattern"+".5_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    
    allfileresultsixtyfivepercentDF.to_csv(outfile+"_pattern"+".65_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    
    allfileresulteightypercentDF.to_csv(outfile+"_pattern"+".8_mincpg"+str(mincpg)+"_CSxOut.txt",sep="\t")
    



In [16]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

12.607500076293945
